# LLaMA4 Fine-Tuning

1. 데이터셋 다운로드

In [1]:
import os
from datasets import load_dataset
dataset = load_dataset("heegyu/namuwiki-extracted" , cache_dir="/datasets/")
print(dataset["train"][0])

namuwiki_20210301_v3.parquet:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/565293 [00:00<?, ? examples/s]

{'title': '!!아앗!!', 'text': '！！ああっと！！\n▲신 세계수의 미궁 2에서 뜬 !!아앗!!\n세계수의 미궁 시리즈에 전통으로 등장하는 대사. 2편부터 등장했으며 훌륭한 사망 플래그의 예시이다.\n세계수의 모험가들이 탐험하는 던전인 수해의 구석구석에는 채취/벌채/채굴 포인트가 있으며, 이를 위한 채집 스킬에 투자하면 제한된 채집 기회에서 보다 큰 이득을 챙길 수 있다. 그러나 분배할 수 있는 스킬 포인트는 한정되어 있기 때문에 채집 스킬에 투자하는 만큼 전투 스킬 레벨은 낮아지게 된다. !!아앗!!이 발생하는 과정을 요약하면 다음과 같다.\n다만 채집 시스템은 신 세계수 시리즈의 그리모어 복제, 복합 채집 스킬인 야생의 감, 5편의 종족 특유 스킬, 크로스의 1레벨이 만렙인 채집 스킬 등으로 편의성이 점차 나아져서 채집 스킬 때문에 스킬 트리가 내려가는 일은 점점 줄어들었다.\n채집용 캐릭터들로 이루어진 약한 파티(ex: 레인저 5명)가 수해에 입장한다.\n필드 전투를 피해 채집 포인트에 도착한 후 열심히 아이템을 캐는 중에...\n!!아앗!!\n라플레시아가 나타났다!\n이때 등장하는 것은 FOE는 아니지만 훨씬 위층에 등장하는 강력한 필드 몬스터이며 선제 공격을 당하게 된다!\n으앙 죽음(hage)\n여담으로 !!아앗!!의 유래는 1인칭 던전 크롤러의 원조 위저드리에서 함정을 건드렸을 때 나오는 대사 Oops!(おおっと！)라고 한다.\n!!아앗!!의 악랄함은 첫 등장한 작품이자 시리즈 중에서도 불친절하기로 정평이 난 2편이 절정이었다. 그야말로 위의 !!아앗!! 시퀀스 그대로, 묻지도 따지지도 않고 채집할 때마다 일정 확률로 강제로 전투에 돌입해야 했다. 게다가 이럴 때 쓰라고 있는 레인저의 스킬 \'위험 감지(중간 확률로 적의 선제 공격을 무효화)\'는 정작 작동하지 않는다!\n참고로 2편에서 채집 도중 !!아앗!!이 뜰 확률은 고작 1%다. 낮아 보이는 확률이어도 플레이 중 한 번이라도 일어나는 것을 경험하는 체감 확률을 고려하여 확률

2. 모델 다운로드

| 구분         | 모델 (Model)                                          | 토크나이저 (Tokenizer)                                              |
| ---------- | --------------------------------------------------- | -------------------------------------------------------------- |
| **역할**     | 입력된 토큰을 기반으로 예측·생성·분류 등을 수행하는 핵심 AI 엔진              | 텍스트를 모델이 이해할 수 있는 토큰 단위로 변환하고, 반대로 토큰을 텍스트로 복원                 |
| **입력/출력**  | 입력: 토큰 시퀀스<br>출력: 확률 분포, 다음 토큰, 임베딩 등               | 입력: 문자열<br>출력: 토큰 ID 시퀀스 및 주석 정보                               |
| **구성 요소**  | 신경망(Transformer, LSTM, CNN 등), 레이어, 파라미터(가중치)로 이루어짐 | 규칙 기반(BPE, WordPiece, SentencePiece 등) 알고리즘과 단어 사전(Vocabulary) |
| **학습 내용**  | 의미, 문맥, 패턴, 언어 구조 등을 학습                             | 자주 나타나는 텍스트 조각 단위를 효율적으로 분리하는 규칙을 학습                           |
| **학습 대상**  | 대규모 텍스트의 문맥적 의미                                     | 텍스트의 통계적 구성(서브워드 단위 빈도 등)                                      |
| **결과물**    | 문장 생성, 번역, 요약, 추론 등 자연어 처리 기능 수행                    | 토큰 리스트 생성 및 디토크나이징                                             |
| **변경 영향**  | 모델 구조/파라미터 변경 시 성능에 직접적인 큰 변화 발생                    | 토크나이저 변경 시 토큰 분포가 바뀌어 모델 성능에 큰 영향 가능                           |
| **독립성 여부** | 토크나이저가 만들어낸 토큰 단위에 의존해야 함                           | 모델 없이도 사용할 수 있지만 대부분 모델과 쌍으로 설계됨                               |


In [ ]:
# transformers 라이브러리를 사용한 모델 다운로드
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import os
from getpass import getpass

model_name = "meta-llama/Llama-4-Scout-17B-16E"

hf_token = os.getenv("HF_TOKEN")  # 먼저 환경변수 확인
if not hf_token:
    print("Hugging Face 토큰을 입력하세요:")
    print("토큰 발급: https://huggingface.co/settings/tokens")
    hf_token = getpass("HF Token: ")

if hf_token:
    login(token=hf_token)
    print("✅ Hugging Face 로그인 성공!")
else:
    print("❌ 경고: 토큰이 입력되지 않았습니다.")

# 모델 다운로드
print(f"모델 다운로드 중: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/models", token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="/models", token=hf_token)

print("다운로드 완료!")
print(f"Tokenizer: {tokenizer}")
print(f"Model: {type(model)}")


✅ Hugging Face 로그인 성공!
모델 다운로드 중: meta-llama/Llama-4-Scout-17B-16E


tokenizer_config.json:   0%|          | 0.00/237k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/106 [00:00<?, ?B/s]

...

model-00001-of-00050.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

model-00002-of-00050.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

model-00007-of-00050.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

model-00003-of-00050.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

model-00005-of-00050.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]